### Question 3

<b>a)</b> To read dataset in a json file and load into Spark DataFrame, I use <b>spark.read.json('<em>path to file</em>')</b>. This code will return Spark DataFrame of dataset which is read from json file.

In [1]:
sdf = spark.read.json('movies.json')

I use <b>printSchema()</b> method of Spark DataFrame to print out the schema of DataFrame.

In [2]:
sdf.printSchema()

root
 |-- cast: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



As we see above:
+ <b>'cast'</b> column has array type and can have null value. Each element in this array has string type or null.
+ <b>'genres'</b> column has array type and can have null value. Each element in this array has string type or null.
+ <b>'title'</b> column has string type and can have null value.
+ <b>'year'</b> column has string type and can have null value.

I use <b>count()</b> method of Spark DataFrame to count number of records in DataFrame.

In [3]:
sdf.count()

28795

As we see above, DataFrame has 28795 records.

<b> b) To see the number of movies released during 2012 and 2015: </b>  
<b>Step 1</b>: I will select year field with <b>select('<em>field name</em>')</b> method of Spark DataFrame. This method will return Spark DataFrame of year field. After that, I use <b>show()</b> method of Spark DataFrame to show the year DataFrame.

In [4]:
year_df = sdf.select("year")
year_df.show()

+----+
|year|
+----+
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1900|
|1901|
|1901|
|1901|
+----+
only showing top 20 rows



<b>Step 2</b>: In year DataFrame, I filter movies released from 2012 to 2015 with <b>filter('SQL Expression string')</b> method of Spark DataFrame. This method return DataFrame after filtering. After that, I use <b>show()</b> method of Spark DataFrame to show the filtered year DataFrame

In [5]:
filtered_year_df = year_df.filter('year >= 2012 AND year <= 2015')
filtered_year_df.show()

+----+
|year|
+----+
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
|2012|
+----+
only showing top 20 rows



<b>Step 3</b>: I use <b>count()</b> method of Spark DataFrame to count number of records in filtered year DataFrame.

In [6]:
filtered_year_df.count()

1015

As we see above, having 1015 movies which are released from 2012 to 2015.

<b>f)</b>

In [7]:
from pyspark.sql.functions import explode, explode_outer, desc, countDistinct

Note:
+ Why I use explode, explode_outer function in pyspark sql functions: Because <b>'cast'</b> column and <b>'genres'</b> column have type array, I want to convert it to rows for being easy to work with names of actors and genres of all movies they participated.  
+ desc function to sort column descending
+ countDistinct function to count distinct value in column

In [8]:
explode_df = sdf.select(explode("cast").alias("actor name"), "genres")

Note:
+ Function <b>explode("<em>name of column wants to be exploded</em>")</b> will return the column contains rows which are converted from each array in column you choose. This function also ignores empty array or null value. 
+ Here I use <b>select()</b> method to create DataFrame which contains <b>'genres'</b> column and <b>'actor name'</b> column (column contains actor names which are converted from each array in <b>'cast'</b> column).

In [9]:
explode_df.show()

+------------------+--------------------+
|        actor name|              genres|
+------------------+--------------------+
|       Paul Boyton|                  []|
|    Ching Ling Foo|             [Short]|
|         May Clark|                  []|
|William Carrington|                  []|
|J. Stuart Blackton|[Short, Crime, Dr...|
| Florence Lawrence|[Short, Crime, Dr...|
|J. Stuart Blackton|[Short, Animated,...|
|   William S. Hart|        [Historical]|
|    William Craven|         [Biography]|
| Florence Lawrence|         [Biography]|
|           Unknown|            [Comedy]|
|   Bertha Regustus|            [Comedy]|
|    Edward Boulden|            [Comedy]|
| Arthur V. Johnson|             [Drama]|
|    Linda Arvidson|             [Drama]|
| Florence Lawrence|                  []|
| William V. Ranous|                  []|
|    Linda Arvidson|            [Comedy]|
|   George Gebhardt|            [Comedy]|
|    Charles Inslee|             [Drama]|
+------------------+--------------

In [10]:
explode_df = explode_df.select("actor name", explode_outer("genres").alias("genres"))

Note:
+ Function <b>explode_outer("<em>name of column wants to be exploded</em>")</b> will return the column contains rows which are converted from each array in column you choose.
+ Why I not choose <b>explode()</b> function as I do with <b>'actor name'</b> column above: Because <b>explode_outer()</b> function does not ignore the emtpy array but returns null value. The actor cannot participate any genres of movies so if we ignore actors who do not participate any genres of movies, sorting will false.
+ Here I use <b>select()</b> method for previous explode DataFrame to create DataFrame which contains <b>'actor name'</b> column and <b>'genres'</b> column (column contains genres which are converted from each array in <b>'genres'</b> column).

In [11]:
explode_df.show()

+------------------+----------+
|        actor name|    genres|
+------------------+----------+
|       Paul Boyton|      null|
|    Ching Ling Foo|     Short|
|         May Clark|      null|
|William Carrington|      null|
|J. Stuart Blackton|     Short|
|J. Stuart Blackton|     Crime|
|J. Stuart Blackton|     Drama|
| Florence Lawrence|     Short|
| Florence Lawrence|     Crime|
| Florence Lawrence|     Drama|
|J. Stuart Blackton|     Short|
|J. Stuart Blackton|  Animated|
|J. Stuart Blackton|  Animated|
|   William S. Hart|Historical|
|    William Craven| Biography|
| Florence Lawrence| Biography|
|           Unknown|    Comedy|
|   Bertha Regustus|    Comedy|
|    Edward Boulden|    Comedy|
| Arthur V. Johnson|     Drama|
+------------------+----------+
only showing top 20 rows



Using <b>groupby("<em>column name</em>")</b> method of Spark DataFrame to group DataFrame by column. This method will return GroupedData object.  
Using <b>agg()</b> method of GroupedData object with <b>countDistinct("<em>column name</em>")</b> function of <b>agg()</b> method to count distint value in one column after group DataFrame by column.

In [12]:
count_df = explode_df.groupby("actor name").agg(countDistinct("genres"))
count_df.show()

+-----------------+-------------+
|       actor name|count(genres)|
+-----------------+-------------+
|     Jean Durrell|            1|
|     Edmund Burns|            5|
|    Peggy Stewart|            6|
| Laurence Olivier|           12|
|      Tura Satana|            1|
|      The Weavers|            1|
|       Gale Storm|            9|
|    Reggie Miller|            3|
|   Caroline Munro|            4|
|     Stephen Root|           11|
| Christine Carère|            1|
|     Teddy Geiger|            1|
|     Oliver Platt|           12|
|Geraldine Chaplin|            8|
| Bradford Dillman|            7|
|   Keir Gilchrist|            3|
|       Snoop Dogg|           10|
| Derrick O'Connor|            1|
|       Lex Barker|            5|
|   James Faulkner|            4|
+-----------------+-------------+
only showing top 20 rows



Use <b>sort(desc("<em>column need to be sorted</em>"))</b> to sort Spark DataFrame descending following column.

In [13]:
count_df.sort(desc("count(genres)")).show()

+------------------+-------------+
|        actor name|count(genres)|
+------------------+-------------+
|               and|           24|
|      Dennis Quaid|           20|
|     Colin Farrell|           18|
|       James Mason|           18|
|      Danny Glover|           18|
|      Gene Hackman|           18|
|      James Coburn|           18|
|     Michael Caine|           18|
|      Michael Peña|           18|
|      Helen Mirren|           18|
|      John Goodman|           17|
|  Lionel Barrymore|           17|
|        Matt Damon|           17|
|    Robert De Niro|           17|
|     James Stewart|           17|
|Laurence Fishburne|           17|
| Samuel L. Jackson|           17|
|    George Sanders|           17|
|     Nicole Kidman|           17|
|       Johnny Depp|           17|
+------------------+-------------+
only showing top 20 rows

